In [1]:
#Downlaod zip
!wget -q https://archive.ics.uci.edu/static/public/22/chess+king+rook+vs+king+pawn.zip
!unzip chess+king+rook+vs+king+pawn.zip

Archive:  chess+king+rook+vs+king+pawn.zip
  inflating: Index                   
  inflating: kr-vs-kp.data           
  inflating: kr-vs-kp.names          


In [22]:
#Import
#!pip install pycaret[full]
import pycaret
from pycaret.classification import *

import pandas as pd

In [15]:
#Get data
data = pd.read_csv('kr-vs-kp.data')

In [16]:
#Setup
s = setup(data, target = 'won', session_id = 123)

,Description,Value
0,Session id,123
1,Target,won
2,Target type,Binary
3,Target mapping,"nowin: 0, won: 1"
4,Original data shape,"(3195, 37)"
5,Transformed data shape,"(3195, 39)"
6,Transformed train set shape,"(2236, 39)"
7,Transformed test set shape,"(959, 39)"
8,Ordinal features,34
9,Categorical features,36


In [23]:
#Compare models
best = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
catboost,CatBoost Classifier,0.9937,0.9993,0.9937,0.9938,0.9937,0.9874,0.9875,3.7530
dt,Decision Tree Classifier,0.9933,0.9932,0.9933,0.9933,0.9933,0.9866,0.9866,0.5780
xgboost,Extreme Gradient Boosting,0.9915,0.9996,0.9915,0.9915,0.9915,0.9830,0.9830,0.4600
lightgbm,Light Gradient Boosting Machine,0.9915,0.9997,0.9915,0.9915,0.9915,0.9830,0.9830,1.2300
rf,Random Forest Classifier,0.9852,0.9988,0.9852,0.9854,0.9852,0.9704,0.9706,0.8550
et,Extra Trees Classifier,0.9848,0.9983,0.9848,0.9850,0.9848,0.9695,0.9697,0.8360
gbc,Gradient Boosting Classifier,0.9808,0.9978,0.9808,0.9813,0.9808,0.9615,0.9620,0.7400
ada,Ada Boost Classifier,0.9660,0.9935,0.9660,0.9664,0.9660,0.9319,0.9322,0.5760
lr,Logistic Regression,0.9624,0.9933,0.9624,0.9629,0.9624,0.9247,0.9251,0.6250
svm,SVM - Linear Kernel,0.9606,0.0000,0.9606,0.9612,0.9606,0.9212,0.9217,0.4300


Processing:   0%|          | 0/69 [00:00<?, ?it/s]

In [28]:
#Prediction
prediction = predict_model(best)
prediction.head()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,CatBoost Classifier,0.9948,0.9995,0.9948,0.9948,0.9948,0.9896,0.9896


,f,f.1,f.2,f.3,f.4,f.5,f.6,f.7,f.8,f.9,...,f.25,f.26,f.27,f.28,t.2,t.3,n.1,won,prediction_label,prediction_score
183,t,f,f,f,t,t,t,f,t,f,...,f,f,f,f,t,t,n,won,won,0.9986
3123,f,f,f,t,t,t,f,f,t,f,...,t,f,f,t,f,t,t,nowin,nowin,0.9950
1877,f,f,f,f,f,t,t,f,f,f,...,f,f,f,f,t,f,n,won,won,0.9553
1440,f,f,f,f,f,t,f,f,f,t,...,f,f,f,f,t,t,n,nowin,nowin,0.9964
535,f,f,f,f,f,f,f,f,f,f,...,f,f,f,f,t,t,n,won,won,0.9662


In [30]:
#Pipeline
save_model(best, 'pipeline')
loaded_best_pipeline = load_model('pipeline')
loaded_best_pipeline

Transformation Pipeline and Model Successfully Saved
Transformation Pipeline and Model Successfully Loaded


Pipeline(memory=FastMemory(location=/tmp/joblib),
         steps=[('label_encoding',
                 TransformerWrapperWithInverse(exclude=None, include=None,
                                               transformer=LabelEncoder())),
                ('numerical_imputer',
                 TransformerWrapper(exclude=None, include=[],
                                    transformer=SimpleImputer(add_indicator=False,
                                                              copy=True,
                                                              fill_value=None,
                                                              keep_empty_features=False,
                                                              missing_values=nan,
                                                              strategy='...
                                    transformer=OneHotEncoder(cols=['n',
                                                                    'f.23'],
                                                              drop_invariant=False,
                                                              handle_missing='return_nan',
                                                              handle_unknown='value',
                                                              return_df=True,
                                                              use_cat_names=True,
                                                              verbose=0))),
                ('clean_column_names',
                 TransformerWrapper(exclude=None, include=None,
                                    transformer=CleanColumnNames(match='[\\]\\[\\,\\{\\}\\"\\:]+'))),
                ('trained_model',
                 <catboost.core.CatBoostClassifier object at 0x7bfce9ee86d0>)],
         verbose=False)

In [31]:
#Normalize setup
s = setup(data, target = 'won', session_id = 123, normalize = True, normalize_method = 'minmax')

,Description,Value
0,Session id,123
1,Target,won
2,Target type,Binary
3,Target mapping,"nowin: 0, won: 1"
4,Original data shape,"(3195, 37)"
5,Transformed data shape,"(3195, 39)"
6,Transformed train set shape,"(2236, 39)"
7,Transformed test set shape,"(959, 39)"
8,Ordinal features,34
9,Categorical features,36


In [34]:
#Train best model -> catboost
cat = create_model('catboost')
cat_results = pull()
create_model('catboost', return_train_score=True)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9911,0.9997,0.9911,0.9911,0.9911,0.9821,0.9821
1,0.9955,0.9994,0.9955,0.9956,0.9955,0.9911,0.9911
2,0.9911,0.9994,0.9911,0.9911,0.9911,0.9821,0.9821
3,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
4,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
5,0.9911,0.9982,0.9911,0.9912,0.9911,0.9821,0.9823
6,0.9955,0.9991,0.9955,0.9956,0.9955,0.9910,0.9911
7,0.9955,1.0000,0.9955,0.9956,0.9955,0.9910,0.9911
8,0.9910,0.9978,0.9910,0.9910,0.9910,0.9820,0.9820


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [41]:
#API
create_api(best, api_name = 'api')
create_docker('api')

API successfully created. This function only creates a POST API, it doesn't run it automatically. To run your API, please run this command --> !python api.py
Writing requirements.txt
Writing Dockerfile
Dockerfile and requirements.txt successfully created.
    To build image you have to run --> !docker image build -f "Dockerfile" -t IMAGE_NAME:IMAGE_TAG .
            


In [44]:
#Finalize best model
final_best = finalize_model(best)
final_best

save_model(best, 'model')
loaded_from_disk = load_model('model')
loaded_from_disk

Transformation Pipeline and Model Successfully Saved
Transformation Pipeline and Model Successfully Loaded


Pipeline(memory=FastMemory(location=/tmp/joblib),
         steps=[('label_encoding',
                 TransformerWrapperWithInverse(exclude=None, include=None,
                                               transformer=LabelEncoder())),
                ('numerical_imputer',
                 TransformerWrapper(exclude=None, include=[],
                                    transformer=SimpleImputer(add_indicator=False,
                                                              copy=True,
                                                              fill_value=None,
                                                              keep_empty_features=False,
                                                              missing_values=nan,
                                                              strategy='...
                                                              verbose=0))),
                ('normalize',
                 TransformerWrapper(exclude=None, include=None,
                                    transformer=MinMaxScaler(clip=False,
                                                             copy=True,
                                                             feature_range=(0,
                                                                            1)))),
                ('clean_column_names',
                 TransformerWrapper(exclude=None, include=None,
                                    transformer=CleanColumnNames(match='[\\]\\[\\,\\{\\}\\"\\:]+'))),
                ('trained_model',
                 <catboost.core.CatBoostClassifier object at 0x7bfd02391930>)],
         verbose=False)

In [45]:
#Experiment
save_experiment('experiment')
exp_from_disk = load_experiment('experiment', data=data)

,Description,Value
0,Session id,123
1,Target,won
2,Target type,Binary
3,Target mapping,"nowin: 0, won: 1"
4,Original data shape,"(3195, 37)"
5,Transformed data shape,"(3195, 39)"
6,Transformed train set shape,"(2236, 39)"
7,Transformed test set shape,"(959, 39)"
8,Ordinal features,34
9,Categorical features,36
